<a href="https://colab.research.google.com/github/PondKann/CXR-Project/blob/main/Test_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###### import

In [107]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
import pandas as pd
from torchsummary import summary
from skimage.io import imread, imsave
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [108]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from imutils import paths
import argparse

from sklearn.model_selection import KFold
from sklearn import metrics
from scipy.stats import zscore

#### Load data

In [109]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [110]:
data_dir = '/content/drive/My Drive/Senior Project/CXR Image3class'

In [111]:
os.listdir(data_dir)

['CA150', 'TB150', 'Normal151', 'All_3class', 'Setting1.xlsx', 'test_model']

#### สร้างตารางเก็บ ชื่อภาพ และชื่อคลาส
ข้อมูลใน All_3class_dir รวมภาพทั้ง 3 คลาส รวม 451 ภาพ TB150, Normal151 และ CA150 (CA คือ lungcancer)

In [112]:
Normal151_dir = os.path.join(data_dir,'Normal151')
TB150_dir = os.path.join(data_dir,'TB150')
CA150_dir = os.path.join(data_dir,'CA150')

All_3class_dir = os.path.join(data_dir,'All_3class')

In [113]:
lungcancer = []
for i in range(len(os.listdir(CA150_dir))):
    img = os.listdir(CA150_dir)[i]
    lungcancer.append(img)

tlungcancer = pd.DataFrame({'ImageName':[lungcancer][0],
                                       'NameType': "Lungcancer" ,
                                       'NumberType': 2                        })

In [114]:
TB700_dir = os.path.join(data_dir,'test_model/Tuberculosis700')
Normal94_dir = os.path.join(data_dir,'test_model/NORMAL94')

AllTest_2class = os.path.join(data_dir,'test_model/AllTest_2class')

In [115]:
#สำหรับ test model
normal = []
for i in range(len(os.listdir(Normal94_dir))):
    img = os.listdir(Normal94_dir)[i]
    normal.append(img)
    
tnormal94 = pd.DataFrame({'ImageName':[normal][0],
                                       'NameType': "Normal" ,
                                       'NumberType': 0                      })

In [116]:
tlungcancer[:10]

,ImageName,NameType,NumberType
0,ca146.jpg,Lungcancer,2
1,ca117.jpg,Lungcancer,2
2,ca140.jpg,Lungcancer,2
3,ca119.jpg,Lungcancer,2
4,ca147.jpg,Lungcancer,2
5,ca126.jpg,Lungcancer,2
6,ca129.jpg,Lungcancer,2
7,ca135.jpg,Lungcancer,2
8,ca144.jpg,Lungcancer,2
9,ca124.jpg,Lungcancer,2


In [117]:
#สำหรับ test model
tb700 = []
for i in range(len(os.listdir(TB700_dir))):
    img = os.listdir(TB700_dir)[i]
    tb700.append(img)

ttb700 = pd.DataFrame({'ImageName':[tb700][0],
                                       'NameType': "Tuberculosis" ,
                                       'NumberType': 1                        })

In [ ]:
data_test2 = pd.concat([tnormal94[:30], ttb700[:30], tlungcancer[:10]])
data_test2

In [119]:
test_NorTB = []
for ImageName in tqdm(data_test2['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test2
    image_path = AllTest_2class +'/' + ImageName                          # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (299, 299))
    test_NorTB.append(img)

100%|██████████| 70/70 [00:04<00:00, 17.24it/s]


In [120]:
data_test = np.array(test_NorTB, dtype="float32") / 255.0

labels_test = np.array(data_test2[['NameType']] )        
labels_test.shape = (len(labels_test),) 

le3_Incep = LabelEncoder()                                                                     
labels_test = le3_Incep.fit_transform(labels_test)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_test = to_categorical(labels_test, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [121]:
le3_Incep.classes_

array(['Lungcancer', 'Normal', 'Tuberculosis'], dtype=object)

In [122]:
labels_test.shape

(70, 3)

In [123]:
data_test.shape

(70, 299, 299, 3)

## dump(save)/load sklearn model

In [130]:
import joblib
import pickle

In [80]:
IncepF_jop= joblib.load('model3_IncepF.joblib')

In [124]:
pred_IncepF_jop = IncepF_jop.predict(data_test, batch_size=8)

In [125]:
print(classification_report(labels_test.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                    pred_IncepF_jop.argmax(axis=1),  
                                    target_names=le3_Incep.classes_))

              precision    recall  f1-score   support

  Lungcancer       0.30      1.00      0.47        10
      Normal       1.00      0.07      0.12        30
Tuberculosis       0.20      0.23      0.22        30

    accuracy                           0.27        70
   macro avg       0.50      0.43      0.27        70
weighted avg       0.56      0.27      0.21        70



In [131]:

IncepF_pkl = pickle.load(open('model_IncepF2.pkl', 'rb'))

In [132]:
pred_IncepF_pkl = IncepF_pkl.predict(data_test, batch_size=8)

In [134]:
print(classification_report(labels_test.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                    pred_IncepF_pkl.argmax(axis=1),  
                                    target_names=le3_Incep.classes_))

              precision    recall  f1-score   support

  Lungcancer       0.30      1.00      0.47        10
      Normal       1.00      0.07      0.12        30
Tuberculosis       0.20      0.23      0.22        30

    accuracy                           0.27        70
   macro avg       0.50      0.43      0.27        70
weighted avg       0.56      0.27      0.21        70



In [135]:
IncepF_pkl2 = pickle.load(open('model_IncepF2.sav', 'rb'))

In [140]:
pred_IncepF_pkl2 = IncepF_pkl2.predict(data_test, batch_size=8)

In [141]:
print(classification_report(labels_test.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                    pred_IncepF_pkl2.argmax(axis=1),  
                                    target_names=le3_Incep.classes_))

              precision    recall  f1-score   support

  Lungcancer       0.30      1.00      0.47        10
      Normal       1.00      0.07      0.12        30
Tuberculosis       0.20      0.23      0.22        30

    accuracy                           0.27        70
   macro avg       0.50      0.43      0.27        70
weighted avg       0.56      0.27      0.21        70



joblib2

In [ ]:
# save the model to disk
joblib.dump(model3_IncepF, 'model_IncepF.sav')

INFO:tensorflow:Assets written to: ram://d0785a86-829d-4bad-9664-8a41b43e78cd/assets


['model_IncepF.sav']

In [ ]:
# load the model from disk
loaded_model = joblib.load('model_IncepF.sav')

result = loaded_model.score(X_test, Y_test)
print(result)

In [ ]:
import pickle

In [ ]:
# save the model to disk
pickle.dump(model3_IncepF, open('model_IncepF2.pkl', 'wb'))

INFO:tensorflow:Assets written to: ram://44e214a6-0132-47f5-a913-16ef8607a88b/assets


In [ ]:
# save the model to disk
pickle.dump(model3_IncepF, open('model_IncepF2.sav', 'wb'))

INFO:tensorflow:Assets written to: ram://a5cf700f-da21-4005-a99c-d4f98b9dbc59/assets


In [ ]:
# load the model from disk
loaded_model = pickle.load(open('model_IncepF2.sav', 'rb'))

result = loaded_model.score(X_test, Y_test)
print(result)

## dump(save)/load sklearn model

In [ ]:
import joblib

In [ ]:
joblib.dump(model3_IncepF, 'model3_IncepF.joblib')  

INFO:tensorflow:Assets written to: ram://ededb2da-3c99-41fc-ae25-33a2dcf58411/assets


['model3_IncepF.joblib']

In [ ]:
IncepF= joblib.load('model3_IncepF.joblib')

In [ ]:
IncepF

joblib2

In [ ]:
# save the model to disk
joblib.dump(model3_IncepF, 'model_IncepF.sav')

INFO:tensorflow:Assets written to: ram://d0785a86-829d-4bad-9664-8a41b43e78cd/assets


['model_IncepF.sav']

In [ ]:
# load the model from disk
loaded_model = joblib.load('model_IncepF.sav')

result = loaded_model.score(X_test, Y_test)
print(result)

In [ ]:
import pickle

In [ ]:
# save the model to disk
pickle.dump(model3_IncepF, open('model_IncepF2.pkl', 'wb'))

INFO:tensorflow:Assets written to: ram://44e214a6-0132-47f5-a913-16ef8607a88b/assets


In [ ]:
# save the model to disk
pickle.dump(model3_IncepF, open('model_IncepF2.sav', 'wb'))

INFO:tensorflow:Assets written to: ram://a5cf700f-da21-4005-a99c-d4f98b9dbc59/assets


In [ ]:
# load the model from disk
loaded_model = pickle.load(open('model_IncepF2.sav', 'rb'))

result = loaded_model.score(X_test, Y_test)
print(result)